In [12]:
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import cv2
import numpy as np
import re

# Paths
pdf_file = r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\scanned_invoice.pdf"
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\poppler-25.07.0\Library\bin"

# Convert PDF → images
pages = convert_from_path(pdf_file, dpi=200, poppler_path=poppler_path)

all_text = ""
for i, page in enumerate(pages, start=1):
    # Convert PIL → OpenCV
    open_cv_image = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)

    # Preprocessing: grayscale + threshold
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # OCR with German language
    text = pytesseract.image_to_string(thresh, lang="deu", config="--oem 3 --psm 6")
    all_text += f"\n--- Page {i} ---\n{text}\n"

# Save raw OCR text
with open(r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\invoice_text_clean.txt", "w", encoding="utf-8") as f:
    f.write(all_text)

print("✅ OCR complete. Cleaned text saved.")


✅ OCR complete. Cleaned text saved.


In [13]:
def extract_invoice_data(text):
    data = {}

    # Store name (e.g., ALDI, REWE, LIDL)
    store_match = re.search(r"(ALDI|REWE|LIDL|EDEKA|NETTO)", text, re.IGNORECASE)
    if store_match:
        data["store"] = store_match.group(1)

    # Date
    date_match = re.search(r"(\d{2}\.\d{2}\.\d{2,4})", text)
    if date_match:
        data["date"] = date_match.group(1)

    # Total amount
    total_match = re.search(r"ZU ZAHLEN\s*([\d,]+\.\d{2})", text)
    if total_match:
        data["total"] = total_match.group(1).replace(",", ".")

    # Payment method
    if "MASTERCARD" in text.upper():
        data["payment_method"] = "MasterCard"
    elif "VISA" in text.upper():
        data["payment_method"] = "Visa"
    elif "BAR" in text.upper():
        data["payment_method"] = "Cash"

    # VAT (MwSt)
    vat_match = re.search(r"MWST.*?(\d{1,2},\d{2})", text)
    if vat_match:
        data["vat"] = vat_match.group(1).replace(",", ".")

    return data

invoice_data = extract_invoice_data(all_text)
print("📑 Extracted Invoice Data:", invoice_data)


📑 Extracted Invoice Data: {'store': 'ALDI', 'date': '05.09.25', 'payment_method': 'MasterCard'}


In [15]:
import pyodbc

# 3️⃣ SQL Server connection parameters
server = r'(localdb)\MSSQLLocalDB'   # or 'localhost\SQLEXPRESS'
database = 'Arsipa'                  # your DB name
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print("✅ Connected to SQL Server")

# 1️⃣ Create table if not exists
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Invoices_1' AND xtype='U')
CREATE TABLE Invoices_1 (
    Id INT IDENTITY(1,1) PRIMARY KEY,
    Store NVARCHAR(100),
    InvoiceDate NVARCHAR(50),
    TotalAmount DECIMAL(10,2),
    PaymentMethod NVARCHAR(50),
    VAT DECIMAL(10,2)
)
""")
conn.commit()

# 2️⃣ Example invoice_data dictionary (from OCR + regex step)
invoice_data = {
    "store": "ALDI",
    "date": "05.09.2025",
    "total": "8.46",
    "payment_method": "MasterCard",
    "vat": "0.55"
}

# 3️⃣ Insert into table
cursor.execute("""
INSERT INTO Invoices_1 (Store, InvoiceDate, TotalAmount, PaymentMethod, VAT)
VALUES (?, ?, ?, ?, ?)
""", (
    invoice_data.get("store"),
    invoice_data.get("date"),
    float(invoice_data.get("total")) if invoice_data.get("total") else None,
    invoice_data.get("payment_method"),
    float(invoice_data.get("vat")) if invoice_data.get("vat") else None
))

conn.commit()
print("✅ Invoice data inserted into SQL Server")

# 4️⃣ Close connection
cursor.close()
conn.close()


✅ Connected to SQL Server
✅ Invoice data inserted into SQL Server


In [17]:
import re
import pyodbc
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import cv2
import numpy as np
import pandas as pd

# ----------------------------
# 1. Configurations
# ----------------------------
pdf_file = r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\scanned_invoice.pdf"
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\poppler-25.07.0\Library\bin"

# SQL Server connection
server = r'(localdb)\MSSQLLocalDB'
database = 'Arsipa'
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'


# ----------------------------
# 2. OCR Function with Preprocessing
# ----------------------------
def ocr_pdf(pdf_path):
    pages = convert_from_path(pdf_path, dpi=200, poppler_path=poppler_path)
    all_text = ""

    for page in pages:
        open_cv_image = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

        text = pytesseract.image_to_string(thresh, lang="deu", config="--oem 3 --psm 6")
        all_text += text + "\n"

    return all_text


# ----------------------------
# 3. Extract Fields from OCR Text
# ----------------------------
def parse_invoice(text):
    data = {}

    # Extract product lines: regex for "NAME PRICE €"
    product_pattern = re.compile(r"([A-ZÄÖÜa-zäöüß\s\-\.]+)\s+(\d+(?:,\d{1,2})?)\s*€")
    products = []

    for match in product_pattern.findall(text):
        name, price = match
        products.append({
            "Product": name.strip(),
            "UnitPrice": float(price.replace(",", ".")),
            "Quantity": 1,   # Default 1, refine later
            "LineTotal": float(price.replace(",", "."))
        })

    data["products"] = products

    # Extract total invoice amount
    total_match = re.search(r"ZU ZAHLEN\s*([\d,]+\.\d{2}|\d+,\d{2})", text)
    if total_match:
        data["InvoiceTotal"] = float(total_match.group(1).replace(",", "."))

    # Extract date
    date_match = re.search(r"(\d{2}\.\d{2}\.\d{2,4})", text)
    if date_match:
        data["InvoiceDate"] = date_match.group(1)

    # Payment channel
    if "MASTERCARD" in text.upper():
        data["PaymentChannel"] = "MasterCard"
    elif "VISA" in text.upper():
        data["PaymentChannel"] = "Visa"
    elif "BAR" in text.upper():
        data["PaymentChannel"] = "Cash"
    else:
        data["PaymentChannel"] = "Unknown"

    # Payment status
    data["PaymentStatus"] = "Success" if "erfolgt" in text.lower() else "Failed"

    return data


# ----------------------------
# 4. Store into SQL Server
# ----------------------------
def save_to_sqlserver(data):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Create tables if not exists
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Invoices' AND xtype='U')
    CREATE TABLE Invoices (
        InvoiceId INT IDENTITY(1,1) PRIMARY KEY,
        InvoiceDate NVARCHAR(50),
        InvoiceTotal DECIMAL(10,2),
        PaymentChannel NVARCHAR(50),
        PaymentStatus NVARCHAR(20)
    )
    """)

    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='InvoiceItems' AND xtype='U')
    CREATE TABLE InvoiceItems (
        ItemId INT IDENTITY(1,1) PRIMARY KEY,
        InvoiceId INT,
        Product NVARCHAR(200),
        Quantity INT,
        UnitPrice DECIMAL(10,2),
        LineTotal DECIMAL(10,2),
        FOREIGN KEY (InvoiceId) REFERENCES Invoices(InvoiceId)
    )
    """)
    conn.commit()

    # Insert invoice
    cursor.execute("""
    INSERT INTO Invoices (InvoiceDate, InvoiceTotal, PaymentChannel, PaymentStatus)
    OUTPUT INSERTED.InvoiceId
    VALUES (?, ?, ?, ?)
    """, (
        data.get("InvoiceDate"),
        data.get("InvoiceTotal"),
        data.get("PaymentChannel"),
        data.get("PaymentStatus")
    ))

    invoice_id = cursor.fetchone()[0]

    # Insert products
    for p in data["products"]:
        cursor.execute("""
        INSERT INTO InvoiceItems (InvoiceId, Product, Quantity, UnitPrice, LineTotal)
        VALUES (?, ?, ?, ?, ?)
        """, (
            invoice_id,
            p["Product"],
            p["Quantity"],
            p["UnitPrice"],
            p["LineTotal"]
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Invoice and items saved to SQL Server")


# ----------------------------
# 5. Run Pipeline
# ----------------------------
ocr_text = ocr_pdf(pdf_file)
invoice_data = parse_invoice(ocr_text)

# Show in Jupyter as DataFrame
df_products = pd.DataFrame(invoice_data["products"])
display(df_products)

print("Invoice Summary:")
print(invoice_data)

# Save to SQL Server
save_to_sqlserver(invoice_data)


,Product,UnitPrice,Quantity,LineTotal
0,ER PACK,0.99,1,0.99
1,x,1.35,1,1.35
2,BELLASAN SONNENBL ÖL,1.69,1,1.69
3,KIDNEY-BOHNEN,0.69,1,0.69
4,ZU ZAHLEN,8.46,1,8.46
5,.... BITTE BELEG AUFBEWAHREN\nKARTENZAHLUNG,8.46,1,8.46


Invoice Summary:
{'products': [{'Product': 'ER PACK', 'UnitPrice': 0.99, 'Quantity': 1, 'LineTotal': 0.99}, {'Product': 'x', 'UnitPrice': 1.35, 'Quantity': 1, 'LineTotal': 1.35}, {'Product': 'BELLASAN SONNENBL ÖL', 'UnitPrice': 1.69, 'Quantity': 1, 'LineTotal': 1.69}, {'Product': 'KIDNEY-BOHNEN', 'UnitPrice': 0.69, 'Quantity': 1, 'LineTotal': 0.69}, {'Product': 'ZU ZAHLEN', 'UnitPrice': 8.46, 'Quantity': 1, 'LineTotal': 8.46}, {'Product': '.... BITTE BELEG AUFBEWAHREN\nKARTENZAHLUNG', 'UnitPrice': 8.46, 'Quantity': 1, 'LineTotal': 8.46}], 'InvoiceTotal': 8.46, 'InvoiceDate': '05.09.25', 'PaymentChannel': 'MasterCard', 'PaymentStatus': 'Success'}
✅ Invoice and items saved to SQL Server


In [9]:
# create Invoices tables
import pyodbc

# Connection
server = r'(localdb)\MSSQLLocalDB'
database = 'Arsipa'
driver = '{ODBC Driver 17 for SQL Server}'
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
cursor = conn.cursor()

# -----------------------------
# 1️⃣ Create Invoices table if not exists
# -----------------------------
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='Invoices')
CREATE TABLE Invoices (
    InvoiceId INT IDENTITY(1,1) PRIMARY KEY,
    InvoiceDate NVARCHAR(50),
    InvoiceTotal DECIMAL(10,2),
    PaymentChannel NVARCHAR(50),
    PaymentStatus NVARCHAR(20)
)
""")
conn.commit()

# -----------------------------
# 2️⃣ Create InvoiceItems table if not exists
# -----------------------------
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='InvoiceItems')
CREATE TABLE InvoiceItems (
    ItemId INT IDENTITY(1,1) PRIMARY KEY,
    InvoiceId INT,
    Product NVARCHAR(200),
    Quantity INT,
    UnitPrice DECIMAL(10,2),
    LineTotal DECIMAL(10,2),
    CONSTRAINT FK_Invoice FOREIGN KEY (InvoiceId) REFERENCES Invoices(InvoiceId)
)
""")
conn.commit()

print("✅ Tables created successfully")
cursor.close()
conn.close()


✅ Tables created successfully
